In [1]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow.keras.models import Model, load_model

from glob import glob
import matplotlib.pyplot as plt


import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

#import h5py
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image
 
#Importing required libraries.
from tensorflow.keras.models import Sequential
# To initialize neural network
from keras.layers import Convolution2D
# Images are two dimensional, concolution step
from keras.layers import MaxPooling2D
# Pooling step
from keras.layers import Flatten
# Convert pools feature map into this large feature vector
from keras.layers import Dense
#To add fully connected layers
from tensorflow.keras.models import Model, load_model
from keras.callbacks import CSVLogger
from tensorflow.keras.optimizers import Adam 

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
IMAGE_SIZE =[224,224]
# BATCH_SIZE = 64
train_dir='/content/drive/MyDrive/Final-Dataset4000-SPLIT/train'
val_dir='/content/drive/MyDrive/Final-Dataset4000-SPLIT/val'         #LEARNING_RATE =0.0005

In [3]:
#add preprocessing layer to the front of VGG

vgg=VGG16(input_shape = IMAGE_SIZE +[3],
          weights = 'imagenet',
          include_top= False)
#By keeping include_top as False we have not specified the last layer in the 
# model.Here, we will add our customized last layers.

58900480/58889256 [==============================] - 0s 0us/step


In [4]:
#dont train the existing weights. The weights are already trained
for layer in vgg.layers:
  layer.trainable =False

In [5]:
folders= glob("/content/drive/MyDrive/Final-Dataset4000-SPLIT/train/*")
#To get all the categories under the train folder path in a list we use a glob method.

In [6]:
x=Flatten()(vgg.output) #Removes last laeyrs

In [7]:
prediction = Dense(len(folders), activation='softmax')(x)
#In prediction, while adding a dense layer we have passed the argument
# len(folders), which states the classes normal and COVID. 
#So, len(folders) are two.

model = Model(inputs=vgg.input, outputs=prediction)

In [8]:
model.summary()
#After specifying the last layer, we will now create an object model. 
#Which has standard VGG input, which we have defined above, 
#and prediction as output.

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
#tell the model what cost and optimization method to use
#Now we will compile the defined model object.
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    #optimizer=optimizers.Adam(lr=LEARNING_RATE),
    metrics=['accuracy']
)

In [10]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range =4, #randomly rotate my image between 0 and 5 degrees 
                                 width_shift_range=0.1,#shift the image along X-axis by 10% 
                                 height_shift_range=0.1, #shift the image along Y-axis by 10% 
                                 shear_range=0.2, # shear the image by 2% 
                                 zoom_range=0.2, #means zoom-in and zoom-out by 2%
                                 fill_mode='constant',
                                 horizontal_flip=True)
train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(224,224),
                                                  batch_size =32,
                                                  class_mode='categorical')
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator= validation_datagen.flow_from_directory(val_dir,
                                                             target_size=(224,224),
                                                             batch_size=32,
                                                             class_mode='categorical')
train_generator.class_indices

Found 5786 images belonging to 2 classes.
Found 2472 images belonging to 2 classes.


{'Covid': 0, 'Non-Covid': 1}

In [11]:
EPOCHS_=5
history= model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    epochs = EPOCHS_,
    steps_per_epoch=len(train_generator),    
    validation_steps=len(validation_generator)
#     batch_size=64,
#    callbacks=callbacks_list
    )

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
181/181 [==============================] - 2487s 14s/step - loss: 0.4874 - accuracy: 0.7874 - val_loss: 0.3767 - val_accuracy: 0.8402
Epoch 2/5
181/181 [==============================] - 111s 611ms/step - loss: 0.3745 - accuracy: 0.8396 - val_loss: 0.7943 - val_accuracy: 0.6824
Epoch 3/5
181/181 [==============================] - 110s 611ms/step - loss: 0.3570 - accuracy: 0.8534 - val_loss: 0.3312 - val_accuracy: 0.8641
Epoch 4/5
181/181 [==============================] - 111s 612ms/step - loss: 0.3119 - accuracy: 0.8737 - val_loss: 0.2792 - val_accuracy: 0.8867
Epoch 5/5
181/181 [==============================] - 111s 612ms/step - loss: 0.2910 - accuracy: 0.8757 - val_loss: 0.4232 - val_accuracy: 0.8256


In [12]:
# save it as a h5 file

import tensorflow as tf

from keras.models import load_model

model.save('model_vgg16.h5')

In [32]:
import weakref

In [33]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

TypeError: ignored

In [31]:
import pickle
with open('covid_detect_model.pkl','wb') as f:
    pickle.dump(model,f)

TypeError: ignored

In [23]:
# Save the Modle to file in the current working directory

Pkl_Filename = "Covid_model_detection.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

TypeError: ignored

In [34]:
import pickle

# Save the trained model as a pickle string.
saved_model = pickle.dumps(model)
# Load the pickled model
model_from_pickle = pickle.loads(saved_model)

TypeError: ignored

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

image_url = input("Enter your image url: ")
print(image_url)

img = image.load_img(image_url,target_size = (224,224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
img_data = preprocess_input(x)

# Use the loaded pickled model to make predictions
class = model_from_pickle.predict(img_data)

print(np.max(class))   # just to see the max value of the classes
print(class)
# print(class[0][1])
if(class[0][0] == np.max(class)):
    print("Covid")
else:
    print("Non-covid")

In [15]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
model = load_model('model_vgg16.h5')

image_url = input("Enter your image url: ")
print(image_url)

img = image.load_img(image_url,target_size = (224,224))
# img = image.load_img('/content/drive/MyDrive/Final-Dataset4000-SPLIT/val/Non-Covid/Normal-10072.png',target_size = (224,224))

x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
img_data = preprocess_input(x)
classes = model.predict(img_data)
print(np.max(classes))   # just to see the max value of the classes
print(classes)
# print(classes[0][1])
if(classes[0][0] == np.max(classes)):
    print("Covid")
else:
    print("Non-covid")

Enter your image url: /content/drive/MyDrive/Final-Dataset4000-SPLIT/val/Non-Covid/Normal-10072.png
/content/drive/MyDrive/Final-Dataset4000-SPLIT/val/Non-Covid/Normal-10072.png
1.0
[[8.109155e-32 1.000000e+00]]
Non-covid


In [16]:
!pip install gradio

     |████████████████████████████████| 3.4 MB 7.4 MB/s 
     |████████████████████████████████| 1.9 MB 73.6 MB/s 
     |████████████████████████████████| 206 kB 71.8 MB/s 
     |████████████████████████████████| 63 kB 3.4 MB/s 
     |████████████████████████████████| 3.0 MB 56.3 MB/s 
     |████████████████████████████████| 961 kB 75.7 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=e4352ec0d23b9e72791a7dc93274030f923d42e33402ca92ead45f33bd93411d
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=7123f3991c219220d1cd4d1e7d40e1a109497dd97e6cb721c2e2b697772bed6c
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


In [ ]:
import gradio as gr
import json
import gradio
from os.path import dirname, realpath, join

In [ ]:
from urllib.request import urlretrieve

# urlretrieve("https://gr-models.s3-us-west-2.amazonaws.com/mnist-model.h5", "mnist-model.h5")
model = tf.keras.models.load_model("model_vgg16.h5")


def recognize_digit(image):
    image = image.reshape(1, -1)
    prediction = model.predict(image).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

im = gradio.inputs.Image(shape=(224, 224), image_mode='L', invert_colors=False, source="canvas")

iface = gr.Interface(
    recognize_digit, 
    im, 
    gradio.outputs.Label(num_top_classes=3),
    live=True,
    interpretation="default",
    capture_session=True,
)

iface.test_launch()

if __name__ == "__main__":
    iface.launch()

Test launch: recognize_digit()... 

ValueError: ignored